In [6]:
import pandas as pd 
from datetime import datetime, date

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [3]:
aggregate = [
    {
        "$match": {
            "session.session_start_time": {"$regex": "^2025"},
            "data_description.modality.abbreviation": {"$nin": ["ecephys", "fib"]},
            "data_description.data_level": "derived",
            "data_description.project_name": "Behavior Platform",
            "procedures": {"$ne": None},
            "$and": [
                {"quality_control.evaluations": {"$exists": True, "$ne": []}},
                {"quality_control.evaluations": {
                    "$not": {
                        "$elemMatch": { "latest_status": { "$ne": "Pass" } }
                    }
                }}
            ]
        }
    },
    {
        "$project": {
            "name": 1,
            "subject_id": "$data_description.subject_id",
            "genotype": "$subject.genotype",
            "date_of_birth": "$subject.date_of_birth",
            "sex": "$subject.sex",
            "session_type": "$session.session_type",
            "session_time": "$session.session_start_time",
            "project_name": "$data_description.project_name", 
            "modality": "$data_description.modality.name",
            "trials_total": "$session.stimulus_epochs.trials_total",
            "trials_rewarded": "$session.stimulus_epochs.trials_rewarded",
        }
    },
    {'$unwind': {'path': '$trials_total', 'preserveNullAndEmptyArrays': False}},
    {'$unwind': {'path': '$trials_rewarded', 'preserveNullAndEmptyArrays': False}},
    {'$unwind': {'path': '$modality', 'preserveNullAndEmptyArrays': False}}
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline=aggregate
)


In [7]:
df = pd.DataFrame(records)
df = df.drop_duplicates(subset='name')

df['session_date'] = df.apply(lambda x: datetime.fromisoformat(x['session_time']).date(), axis=1)
df['session_time'] = df.apply(lambda x: datetime.fromisoformat(x['session_time']).time(), axis=1)
df['date_of_birth'] = df.apply(lambda x: datetime.strptime(x['date_of_birth'], '%Y-%m-%d').date(), axis=1)
df['age'] = df.apply(lambda x: (x['session_date'] - x['date_of_birth']).days, axis=1)

order = ['project_name','name','subject_id','genotype','date_of_birth','sex','modality','session_type','session_date','age','session_time','trials_total','trials_rewarded']
df = df[order]
df

,project_name,name,subject_id,genotype,date_of_birth,sex,modality,session_type,session_date,age,session_time,trials_total,trials_rewarded
0,Behavior Platform,behavior_761432_2025-03-03_11-51-23_processed_...,761432,Dbh-Cre-KI/wt,2024-08-21,Male,Behavior,Coupled Baiting,2025-03-03,194,11:51:23.846831,613,246
1,Behavior Platform,behavior_761433_2025-03-03_11-47-13_processed_...,761433,Dbh-Cre-KI/wt,2024-08-21,Male,Behavior,Coupled Without Baiting,2025-03-03,194,11:47:13.392064,509,209
2,Behavior Platform,behavior_770803_2025-02-28_13-11-33_processed_...,770803,Ai230(TIT2L-XCaMPG-WPRE-ICL-ChRmine-oScarlet-I...,2024-10-07,Male,Behavior,Uncoupled Baiting,2025-02-28,144,13:11:33.083835,539,261
3,Behavior Platform,behavior_781469_2025-03-03_13-26-23_processed_...,781469,wt/wt,2024-10-08,Male,Behavior,Uncoupled Baiting,2025-03-03,146,13:26:23.690824,403,180
4,Behavior Platform,behavior_752014_2025-03-03_09-34-48_processed_...,752014,Dbh-Cre-KI/wt,2024-07-05,Male,Behavior,Uncoupled Without Baiting,2025-03-03,241,09:34:48.766920,651,333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,Behavior Platform,behavior_758018_2025-02-18_09-19-22_processed_...,758018,Dbh-Cre-KI/wt,2024-08-04,Female,Behavior,Uncoupled Without Baiting,2025-02-18,198,09:19:22.803530,559,297
467,Behavior Platform,behavior_777098_2025-04-22_18-17-36_processed_...,777098,Drd1a-Cre/wt;Adora2a-T2A-FlpO/wt;Ai233(TICL-Ch...,2024-11-11,Female,Behavior,Uncoupled Baiting,2025-04-22,162,18:17:36.949337,466,161
468,Behavior Platform,behavior_777098_2025-04-21_16-01-17_processed_...,777098,Drd1a-Cre/wt;Adora2a-T2A-FlpO/wt;Ai233(TICL-Ch...,2024-11-11,Female,Behavior,Uncoupled Baiting,2025-04-21,161,16:01:17.308796,460,210
469,Behavior Platform,behavior_774390_2025-06-25_18-42-52_processed_...,774390,Drd3-Cre_KI196/wt,2024-10-26,Male,Behavior,Uncoupled Baiting,2025-06-25,242,18:42:52.752822,496,231


In [8]:
df.to_csv('/data/metadata/dynamic_foraging_metadata.csv', index= False)